In [21]:
# For compatibility across multiple platforms
import os
import numpy as np
import pandas as pd
from scipy import spatial


# Load files using DictReader in Python
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn import cluster
from sklearn import preprocessing
from sklearn.model_selection import KFold
import sklearn


from ultimate.mlp import MLP 

from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
from keras.layers import BatchNormalization

In [2]:
train_filepath = "pubg-finish-placement-prediction/train_V2_clean.csv"

#trainset_file = open(train_filepath,'rU')
trainset = pd.read_csv(train_filepath,index_col=0)

/anaconda2/lib/python2.7/site-packages/numpy/lib/arraysetops.py:466: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
print(len(trainset))

4446965


In [4]:
def feature_engineering_two(moddedTrain):
    moddedTrain['playersJoined'] = moddedTrain.groupby('matchId')['matchId'].transform('count')
    moddedTrain['killsNorm'] = moddedTrain['kills']*((100-moddedTrain['playersJoined'])/100 + 1)
    moddedTrain['damageDealtNorm'] = moddedTrain['damageDealt']*((100-moddedTrain['playersJoined'])/100 + 1)
    moddedTrain['maxPlaceNorm'] = moddedTrain['maxPlace']*((100-moddedTrain['playersJoined'])/100 + 1)
    moddedTrain['matchDurationNorm'] = moddedTrain['matchDuration']*((100-moddedTrain['playersJoined'])/100 + 1)
    moddedTrain['healsandboosts'] = moddedTrain['heals'] + moddedTrain['boosts']
    moddedTrain['totalDistance'] = moddedTrain['rideDistance'] + moddedTrain['walkDistance'] + moddedTrain['swimDistance']
    moddedTrain['killsWithoutMoving'] = ((moddedTrain['kills'] > 0) & (moddedTrain['totalDistance'] == 0))
    moddedTrain['headshot_rate'] = moddedTrain['headshotKills'] / moddedTrain['kills']
    moddedTrain['headshot_rate'] = moddedTrain['headshot_rate'].fillna(0)
    moddedTrain.drop(moddedTrain[moddedTrain['killsWithoutMoving'] == True].index, inplace=True)
    moddedTrain.drop(moddedTrain[moddedTrain['roadKills'] > 8].index, inplace=True)
    

In [17]:
def feature_engineering(trainset,is_train=True):
    # When this function is used for the training data, load train_V2.csv :
    if is_train: 
        print("processing train_V2.csv")
        #df = pd.read_csv("pubg-finish-placement-prediction/train_V2_clean.csv",index_col=0)
        df = trainset
        # Only take the samples with matches that have more than 1 player 
        # there are matches with no players or just one player ( those samples could affect our model badly) 
        df = df[df['maxPlace'] > 1]
    
    # When this function is used for the test data, load test_V2.csv :
    else:
        print("processing test_V2.csv")
        df = pd.read_csv(INPUT_DIR + 'test_V2.csv')
        
    # Make a new feature indecating the total distance a player cut :
 
    df['totalDistance'] = df['rideDistance'] + df["walkDistance"] + df["swimDistance"]

    # Process the 'rankPoints' feature by replacing any value of (-1) to be (0) :
    df['rankPoints'] = np.where(df['rankPoints'] <= 0 ,0 , df['rankPoints'])
                           
    

    target = 'winPlacePerc'
    # Get a list of the features to be used
    features = list(df.columns)
    
    # Remove some features from the features list :
    features.remove("Id")
    features.remove("matchId")
    features.remove("groupId")
    features.remove("matchDuration")
    features.remove("matchType")
    
    y = None
    
    # If we are processing the training data, process the target
    # (group the data by the match and the group then take the mean of the target) 
    if is_train: 
        y = np.array(df.groupby(['matchId','groupId'])[target].agg('mean'), dtype=np.float64)
        # Remove the target from the features list :
        features.remove(target)
    
    # Make new features indicating the mean of the features ( grouped by match and group ) :
    print("get group mean feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('mean')
    # Put the new features into a rank form ( max value will have the highest rank)
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    
    # If we are processing the training data let df_out = the grouped  'matchId' and 'groupId'
    if is_train: df_out = agg.reset_index()[['matchId','groupId']]
    # If we are processing the test data let df_out = 'matchId' and 'groupId' without grouping 
    else: df_out = df[['matchId','groupId']]
    
    # Merge agg and agg_rank (that we got before) with df_out :
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_mean", "_mean_rank"], how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the max value of the features for each group ( grouped by match )
    print("get group max feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('max')
    # Put the new features into a rank form ( max value will have the highest rank)
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    # Merge the new (agg and agg_rank) with df_out :
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_max", "_max_rank"], how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the minimum value of the features for each group ( grouped by match )
    print("get group min feature")
    agg = df.groupby(['matchId','groupId'])[features].agg('min')
    # Put the new features into a rank form ( max value will have the highest rank)
    agg_rank = agg.groupby('matchId')[features].rank(pct=True).reset_index()
    
    # Merge the new (agg and agg_rank) with df_out :
    df_out = df_out.merge(agg.reset_index(), suffixes=["", ""], how='left', on=['matchId', 'groupId'])
    df_out = df_out.merge(agg_rank, suffixes=["_min", "_min_rank"], how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the number of players in each group ( grouped by match )
    print("get group size feature")
    agg = df.groupby(['matchId','groupId']).size().reset_index(name='group_size')
     
    # Merge the group_size feature with df_out :
    df_out = df_out.merge(agg, how='left', on=['matchId', 'groupId'])
    
    # Make new features indicating the mean value of each features for each match :
    print("get match mean feature")
    agg = df.groupby(['matchId'])[features].agg('mean').reset_index()
    
    # Merge the new agg with df_out :
    df_out = df_out.merge(agg, suffixes=["", "_match_mean"], how='left', on=['matchId'])
    
    # Make new features indicating the number of groups in each match :
    print("get match size feature")
    agg = df.groupby(['matchId']).size().reset_index(name='match_size')
    
    # Merge the match_size feature with df_out :
    df_out = df_out.merge(agg, how='left', on=['matchId'])
    
    # Drop matchId and groupId
    df_out.drop(["matchId", "groupId"], axis=1, inplace=True)
    
    y = y.tolist()
    

    return df_out,y

In [19]:
feature_engineering_two(trainset)
df_out,y = feature_engineering(trainset)

processing train_V2.csv
get group mean feature
get group max feature
get group min feature
get group size feature
get match mean feature
get match size feature


In [7]:
x_train, x_val, y_train, y_val = train_test_split(df_out, y , test_size=0.2)

# SET UP FOR MODEL ONE

## Select the trainset features for tree one 

In [8]:
x_one = x_train
y_one = y_train

x_one_val = x_val
y_one_val = y_val


#set the dmatrix
#trainset_one_dmatrix = xgb.DMatrix(x_one.values,label=y_train.values,feature_names=x_one.columns)
#valset_one_dmatrix = xgb.DMatrix(x_one_val.values,label=y_one_val.values,feature_names=x_one_val.columns)

In [53]:
trainset_one_dmatrix = xgb.DMatrix(x_one.values,label=y_train,feature_names=x_one.columns)
valset_one_dmatrix = xgb.DMatrix(x_one_val.values,label=y_one_val,feature_names=x_one_val.columns)

In [10]:
x_one.head()

,assists_mean,boosts_mean,damageDealt_mean,DBNOs_mean,headshotKills_mean,heals_mean,killPlace_mean,killPoints_mean,kills_mean,killStreaks_mean,...,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,totalDistance,match_size
240822,1.0,2.750000,245.205000,0.5,0.25,5.25,29.0,1334.250000,0.5,0.500000,...,1790.259341,0.010989,1.617575,0.010989,0.032967,1452.990000,3.791209,1508.857143,3244.866915,91
1041049,1.0,1.000000,204.793333,1.0,0.00,1.00,41.0,1389.666667,1.0,0.666667,...,41.771429,0.000000,3.151837,0.010204,0.000000,1027.133776,3.673469,1508.500000,1072.057041,98
1677229,0.0,0.333333,101.013333,1.0,0.00,0.00,89.0,1248.666667,0.0,0.000000,...,57.019175,0.000000,0.316598,0.030928,0.010309,1212.426598,3.051546,1518.762887,1269.762371,97
1378932,0.0,0.000000,50.000000,0.5,0.00,0.00,71.0,0.000000,0.5,0.500000,...,69.852041,0.000000,2.222857,0.000000,0.000000,936.041653,3.030612,0.000000,1008.116551,98
2024780,0.2,1.800000,285.380000,1.6,0.20,7.20,21.2,1300.600000,2.2,1.000000,...,1647.362366,0.000000,0.000000,0.032258,0.000000,1801.833548,4.204301,1495.451613,3449.195914,93


## Configure tree one

In [54]:
params = {
    "max_depth" : 7,
    "eval_metric" : ["mae"],
    "lambda": 1.1,   
}


In [55]:
iters = 30

tree_one = xgb.train(params, trainset_one_dmatrix, evals=[(trainset_one_dmatrix, "train"),(valset_one_dmatrix, 'val')], num_boost_round = iters)

[15:42:07] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 252 extra nodes, 0 pruned nodes, max_depth=7
[0]	train-mae:0.184906	val-mae:0.184939
[15:42:58] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 254 extra nodes, 0 pruned nodes, max_depth=7
[1]	train-mae:0.134033	val-mae:0.134024
[15:43:52] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 252 extra nodes, 0 pruned nodes, max_depth=7
[2]	train-mae:0.099873	val-mae:0.099852
[15:44:42] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 252 extra nodes, 0 pruned nodes, max_depth=7
[3]	train-mae:0.077337	val-mae:0.077323
[15:45:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 250 extra nodes, 0 pruned nodes, max_depth=7
[4]	train-mae:0.062542	val-mae:0.062545
[15:46:27] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 252 extra nodes, 0 pruned nodes, max_depth=7
[5]	train-mae:0.052687	val-mae:0.052702
[15:47:22] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 250 extra node

xgb.plot_importance(tree_one,importance_type='cover')
xgb.plot_importance(tree_one,importance_type='weight')

# SET UP FOR MODEL TWO

## Select the trainset features for tree two

In [13]:
x_two = x_train
y_two = y_train

x_two_val = x_val
y_two_val = y_val

#scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1), copy=False).fit(x_two)

In [41]:
#scaler.transform(x_two)

array([[-1.        , -1.        , -0.98992342, ...,  0.920351  ,
        -0.69493471,  0.95789474],
       [-0.96825397, -0.95959596, -0.92319629, ..., -1.        ,
        -0.6123272 ,  0.91578947],
       [-1.        , -0.87878788, -0.95815599, ...,  0.93947277,
        -0.37598127,  0.91578947],
       ...,
       [-1.        , -0.96969697, -0.98488513, ..., -1.        ,
        -0.74718103,  0.87368421],
       [-0.97619048, -1.        , -0.97965538, ...,  0.93687894,
        -0.71874774,  0.87368421],
       [-1.        , -0.90909091, -0.98606409, ...,  0.93673303,
        -0.43835967,  0.89473684]])

## Define the NN

In [47]:
# create NN_model
NN_model = Sequential()
NN_model.add(Dense(x_two.shape[1],  input_dim = x_two.shape[1], activation='relu'))
NN_model.add(Dense(136, activation='relu'))
NN_model.add(BatchNormalization())
NN_model.add(Dense(136, activation='relu'))
NN_model.add(Dense(136, activation='relu'))
NN_model.add(Dense(136, activation='relu'))
NN_model.add(Dense(100, activation='relu'))
NN_model.add(Dense(100, activation='relu'))
NN_model.add(Dense(100, activation='relu'))
NN_model.add(Dense(100, activation='relu'))
NN_model.add(Dense(80, activation='relu'))
NN_model.add(Dense(80, activation='relu'))






# output Layer
NN_model.add(Dense(1, activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_73 (Dense)             (None, 170)               29070     
_________________________________________________________________
dense_74 (Dense)             (None, 136)               23256     
_________________________________________________________________
batch_normalization_31 (Batc (None, 136)               544       
_________________________________________________________________
dense_75 (Dense)             (None, 136)               18632     
_________________________________________________________________
dense_76 (Dense)             (None, 136)               18632     
_________________________________________________________________
dense_77 (Dense)             (None, 136)               18632     
_________________________________________________________________
dense_78 (Dense)             (None, 100)               13700     
__________

In [40]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [48]:
NN_model.fit(x=x_two, y=y_two, batch_size=1000,
             epochs=5, verbose=1, callbacks=callbacks_list,
             validation_split=0.20, validation_data=None, shuffle=True,
             class_weight=None, sample_weight=None, initial_epoch=0,
             steps_per_epoch=None, validation_steps=None)


Train on 1297116 samples, validate on 324279 samples
Epoch 1/5
1297116/1297116 [==============================] - 51s 40us/step - loss: 0.0671 - mean_absolute_error: 0.0671 - val_loss: 0.0573 - val_mean_absolute_error: 0.0573

Epoch 00001: val_loss did not improve from 0.04533
Epoch 2/5
1297116/1297116 [==============================] - 37s 28us/step - loss: 0.0528 - mean_absolute_error: 0.0528 - val_loss: 0.0630 - val_mean_absolute_error: 0.0630

Epoch 00002: val_loss did not improve from 0.04533
Epoch 3/5
1297116/1297116 [==============================] - 37s 28us/step - loss: 0.0489 - mean_absolute_error: 0.0489 - val_loss: 0.0673 - val_mean_absolute_error: 0.0673

Epoch 00003: val_loss did not improve from 0.04533
Epoch 4/5
1297116/1297116 [==============================] - 36s 28us/step - loss: 0.0465 - mean_absolute_error: 0.0465 - val_loss: 0.0535 - val_mean_absolute_error: 0.0535

Epoch 00004: val_loss did not improve from 0.04533
Epoch 5/5
1297116/1297116 [====================

# ENSENBLE MODELS ON VALSET

### Predict - Update - Predict

In [81]:
def gen_predictions(models,labels,num_samples):
    labels = labels[:num_samples]
    
    predictions = []
    
    for model in models:
        data = model['data']
        predictor = model['predictor']

        if model['type'] == "tree":
            matrix = xgb.DMatrix(data,feature_names=data.columns)
            prediction = predictor.predict(matrix)
            predictions.append(prediction)
            print("tree")
        if model['type'] == "nn":
            prediction = predictor.predict(data)
            predictions.append(prediction)
            print("nn")
        else:
            continue

        
        
    print(len(predictions[0]))
    print(len(predictions[1]))

    avg_predictions = [0.0]*len(predictions[0])
    weight = 0
    for j in range(len(predictions[0])):
        if predictions[0][j] - predictions[1][j] <= .0001:
            avg_predictions[j] = 1.0 * (predictions[0][j] + predictions[1][j]) / 2.0
        else:
            avg_predictions[j] = predictions[0][j]

    print(len(avg_predictions))
    return avg_predictions
            

### Ensemble model predictions

In [82]:
models = []
models.append({'data':x_one_val,
                'predictor':NN_model,
                'type':"nn"})

models.append({'data':x_one_val,
                'predictor':tree_one,
                'type':"tree"})

labels = y_val

num_samples = 200000

predictions = gen_predictions(models,labels,num_samples)

nn
tree
405349
405349
405349


In [84]:
print(sklearn.metrics.mean_absolute_error(labels,predictions))

0.042158966795849594


In [85]:
print(sklearn.metrics.mean_absolute_error(labels,tree_one.predict(valset_one_dmatrix)))

0.032768144979863266
